In [1]:
import time

import IPython
from IPython.display import display, HTML

# import pywt
# import pywt.data

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import *
from md import *

%matplotlib inline

import math
import numpy as np

import matplotlib.gridspec as gridspec
#-------------------------------------------------------------------------------------#
mpl.use("nbAgg")
plt.style.use(['seaborn-v0_8-notebook'])
# plt.style.use(['seaborn-notebook'])
plt.rcParams.update({"figure.facecolor": (0,0,0,0),
                     "figure.dpi": 90,
                     "animation.html": "jshtml",
                     "animation.embed_limit": 100})
plt.rcParams['font.size'] = 20

# Задание различных физических констант
z = 150 # Красное смещение
k = 1.38e-23 # Постоянная Больцмана Дж/К
m_p = 1.67e-27 # Масса протона кг
pk_to_meter = 3e16 # Переводной коэффициент пк->м
G = 6.67e-11 # гравитационная постоянная Н*м^2/кг^2
M_pbh = 1e9 # масса черной дыры в солнечных массах
M_sun = 2e30 # масса солнца в кг
gamma = 5/3 # коэффициент адиабаты
t_end_years = 1e9 # Конечное время симуляции в годах
t_end_seconds = 3.154e7 * t_end_years
draw_step = 10000 # Шаг для отрисовки

# Множители для отладки
PP = 1  #множитель перед членом с давлением
GG = 1  #множитель перед членом с гравитацией

# Средняя плотность Вселенной
Rho_mean = 8.5e-27 * (5.1e-5 * (1+z)**4 + 0.308 * (1+z)**3 + 0.691 + 0.000949 * (1+z)**2) # в кг/м**3

# Блок констант далее требуется для условия Куранта-Фридрихсона-Леви (КФЛ)
mu = 1 # Средний молекулярный вес, это значение для водорода, если будем что-то добавлять то будет что-то другое
a1 = 1/119  # первый коэффициент в формуле для температуры
a2 = 1/115  # второй коэффициент в формуле для температуры
TT = 2.7 * (1+z) * (1 + a1**(-1)*(1+z)**(-1)/(1+a2**(3/2)*(1+z)**(3/2))) ** (-1)
a = np.sqrt(k*TT / (mu*m_p))   #скорость звука в м/c


# Радиус остановки DM
Rho_i = 4.5*10**35 #  плотность в кг/пк**3
i = 0
t = ((1+10000)/(1+z))**(3/2) * 52000 #+ dt*i  # время в годах
r_s = 0.553 * (3*M_pbh*M_sun/(4*np.pi*Rho_i))**(1/3) * ((t-52000)/52000)**(8/9)  # радиус остановки в пк

# Создаем одномерную пространственную сетку


# Диапазон сетки
r_min = 1 # внутренняя граница сетки в парсеках
r_max = 500 # внешняя граница сетки в парсеках


# Логарифмическая сетка
# r_phys - физические координаты узлов сетки
# r_log - десятичные логарифмы координат сетки (новая переменная)
r_log_min = np.log(r_min)
r_log_max = np.log(r_max)
length_r = 100
r_log = np.linspace(r_log_min, r_log_max, length_r)
# r_phys = np.logspace(r_log_min, r_log_max, length_r)
r_phys = np.exp(1) ** r_log
# r_log = np.log10(r_phys)
log_step = r_log[1]-r_log[0]

# Равномерная сетка
# dr = 0.01 # шаг сетки в парсеках

# Количество узлов сетки
# length_r = int((r_max - r_min) / dr) + 1

# Массив координат узлов сетки
# r = np.array([r_min + i * dr for i in range(length_r)])

# Задание первичного (пробного) условия КФЛ
# Далее условие будет пересчитываться

# Вид условия: С = ((a + |u_x|) * dt)/dx < 0.5
# Здесь a-эффективная скорость звука, u_x-скорость газа, dt и dx шаги по времени и пространству
# Константа в условии КФЛ
C = 0.1


# Для логарифмической сетки
# Берем самое начало сетки, так как dt здесь будет минимально
dt = C * (r_phys[1]-r_phys[0]) * pk_to_meter / a


# Для равномерной сетки.
# Здесь мы ставим условие на dt, чтобы у нас не ехали узлы сетки,
# При учете скорости нужно брать минимальное значение dt из всей сетки
# Парсеки здесь переведены в метры, dt в секундах
# dt = C * (dr*3e16) / a

In [2]:
# Создаем массивы для хранения различных переменных
U = np.zeros((2, length_r)) # скорость газа U_0 = 0
Rho = np.zeros((2, length_r)) # плотность газа Rho
P = np.zeros((2, length_r)) # давление газа P
T = np.zeros(length_r) # температура газа T
Eps = np.zeros(length_r) # внутренняя энергия газа U
E = np.zeros((2, length_r)) # Полная энергия
n_H = np.zeros(length_r) # количественная плотность водорода
Phi = np.zeros(length_r)             # гравитационный потенциал

r_log_temp_2 = np.append(r_log[0] - log_step, r_log)
r_log_temp_2 = np.append(r_log_temp_2, r_log[-1] + log_step)
r_temp_2 = np.exp(1) ** r_log_temp_2
Phi_temp = (-G*mdmt(r_temp_2[0]/r_s)/(r_temp_2[0] * pk_to_meter) * (Rho_mean * 4*np.pi/3 * (r_s*pk_to_meter)**3)
              -G*M_pbh*M_sun/(r_temp_2[0] * pk_to_meter)) # в м**2/с**2
Phi_temp_2 = (-G*mdmt(r_temp_2[-1]/r_s)/(r_temp_2[-1] * pk_to_meter) * (Rho_mean * 4*np.pi/3 * (r_s*pk_to_meter)**3)
              -G*M_pbh*M_sun/(r_temp_2[-1] * pk_to_meter)) # в м**2/с**2

#начальные условия
for j in range(length_r):
    # Гравитационный потенциал
    Phi[j] = (-G*mdmt(r_phys[j]/r_s)/(r_phys[j] * pk_to_meter) * (Rho_mean * 4*np.pi/3 * (r_s*pk_to_meter)**3)
              -G*M_pbh*M_sun/(r_phys[j] * pk_to_meter)) # в м**2/с**2
#     print(mdmt((r_phys[j]/r_s))* (Rho_mean * 4*np.pi/3 * (r_s*pk_to_meter)**3))

    # Температура
    T[j] = 2.7 * (1+z) * (1 + a1**(-1)*(1+z)**(-1)/(1+a2**(3/2)*(1+z)**(3/2)))**(-1)  # в кельвинах


    # Плотность барионов
    omega_b = 0.049     # эти значения скорее всего должны быть другие
    omega_m = 0.308
    Rho[0, j] = omega_b * rrb(r_phys[j]/r_s)/omega_m    #пробная плотность в Rho_mean

    # Так как переопределили Rho и оставили метры то нужно переопределить единицы массы
    M_unit = Rho_mean   # Коэффициент перевода единиц массы в кг


    # Концентрация водорода
    n_H[j] = (Rho[0, j])/(m_p/M_unit)


    # массовая плотность внутренней энергия
    Eps[j] = (n_H[j] * 3/2 * k * T[j])/(Rho[0, j])  # В Дж/M_unit


    # Скорости
    U[0, j] = 0   # В м/c


    # Полная энергия
    E[0, j] = (Rho[0, j]) * (U[0, j]**2/2 + Eps[j]) # В M_unit/м*с**2


    # Уравнение состояния
    P[0, j] = (gamma - 1) * (Rho[0, j]) * Eps[j]   # Давление в M_unit/м*с**2


Phi = np.append(Phi_temp, Phi)
Phi = np.append(Phi, Phi_temp_2)
P[0, -1] = P[0, -2]
U[0, -1] = U[0, -2]

# Расчет решения


# Вычисление значений массивов S, V, S_dual и V_dual (площади и обьемы шаровых слоев)
# S_2 и V_2 - вспомогательные массивы отличающиеся от оригинальных количеством элементов
S = np.zeros(length_r)
V = np.zeros(length_r)
S_dual = np.zeros(length_r)
V_dual = np.zeros(length_r)

r_log_temp = np.append(r_log, r_log[-1] + log_step)
r_temp = np.exp(1) ** r_log_temp

S = 4 * np.pi * (r_phys[1:]**2-r_phys[:-1]**2) * pk_to_meter**2
S_2 = np.append(S,(4 * np.pi * ((r_phys[-1]+(r_temp[-1]-r_temp[-2]))**2 - r_phys[-1]**2) * pk_to_meter**2))
V = 4/3 * np.pi * (r_phys[1:]**3-r_phys[:-1]**3) * pk_to_meter**3
V_2 = np.append(V,(4/3 * np.pi * ((r_phys[-1]+(r_temp[-1]-r_temp[-2]))**3 - r_phys[-1]**3) * pk_to_meter**3))
S_dual = 4 * np.pi * ((r_phys[1:] + (r_temp[2:]-r_temp[1:-1]) / 2)**2 - (r_phys[:-1] + (r_phys[1:]-r_phys[:-1]) / 2)**2) * pk_to_meter**2
V_dual = 4/3 * np.pi * ((r_phys[1:] + (r_temp[2:]-r_temp[1:-1]) / 2)**3 - (r_phys[:-1] + (r_phys[1:]-r_phys[:-1]) / 2)**3) * pk_to_meter**3


du_dr = np.zeros(length_r)
U_half = np.zeros(length_r)

# Задание начального значения для i
i = 0

# Задание начального значения для dt_check (проверочный массив шагов по времени)
dt_check = np.zeros(1)
dt_check[0] = dt

U_draw = U[0, :]
Rho_draw = Rho[0, :]
E_draw = E[0, :]
sim_time = 0
time_0 = time.time()

dt_minimizer = 0.0009
dt_minimizer_2 = 0.001

while (sim_time < t_end_seconds):
    if (i == 0):
        i = 1

    # Лагранжев шаг

    # Вычисление скорости на полшага вперед по времени
    dP_dx = (P[0,2:] - P[0,:-2]) / (2 * log_step)
    dPhi_dx = (Phi[2:] - Phi[:-2]) / (2 * log_step)
    # Необходимо проверить все производные
    # Скорость центрована по узлам сетки а плотность и энергия по центру ячеек
    U_half[1:-1] = U[0,1:-1] + (- PP*(dt * dP_dx/(r_phys[1:-1] * pk_to_meter)) / (Rho[0,1:-1])
                                - GG*(dt * dPhi_dx[1:-1]/(r_phys[1:-1] * pk_to_meter)))
    U_half[-1] = U_half[-2]
    U_half[0] = U_half[1]
    # Вычисление производной скорости по времеkни
    du_dx = (U_half[2:] - U_half[:-2]) / (2 * log_step)
    du_dr[1:-1] = du_dx / (r_phys[1:-1] * pk_to_meter)
    du_dr[-1] = du_dr[-2]
    du_dr[0] = du_dr[1]

    # Промежуточная полная энергия
    E_inter = E[0,:] - P[0,:]*du_dr*dt - Rho[0,:] * U_half[:] * dPhi_dx/(r_phys * pk_to_meter) * dt

    # Шаг адвекции

    # Вычисление скорости
    U[1,:-1] = U[0,:-1] + (dt * U_half[:-1] * np.abs(U_half[:-1]) * S_dual) / V_dual
    U[1,-1] = U[1,-2]

    # Вычисление плотности
    Rho[1,1:] = Rho[0,1:] - np.abs((dt * Rho[0,1:] * U_half[1:] * S) / V)
    Rho[1,0] = Rho[0,0] - np.abs((dt * Rho[0,0] * U_half[0] * S[0]) / V[0]) * (U_half[0]>0)
    Rho[1,:-1] = Rho[1,:-1] + np.abs((dt * Rho[0,:1] * U_half[1:] * S_2[1:]) / V_2[1:]) * np.where(U_half[1:]<0, 1, 0)
    Rho[1,1:] = Rho[1,1:] + np.abs((dt * Rho[0,:-1] * U_half[:-1] * S) / V) * np.where(U[1,:-1]>0, 1, 0)

    # Вычисление полной энергии
    E[1,1:] = E[0,1:] - np.abs((dt * E_inter[1:] * U_half[1:] * S) / V)
    # print('EEE', E[0,:])
#     print('reee', np.abs((dt * E_inter[1:] * U_half[1:] * S) / V))
    E[1,0] = E[0,0] - np.abs((dt * E_inter[0] * U_half[0] * S[0]) / V[0]) * (U_half[0]>0)
    E[1,:-1] = E[1,:-1] + np.abs((dt * E_inter[1:] * U_half[1:] * S_2[1:]) / V_2[1:]) * np.where(U_half[1:]<0, 1, 0)
    E[1,1:] = E[1,1:] + np.abs((dt * E_inter[:-1] * U_half[:-1] * S) / V) * np.where(U_half[:-1]>0, 1, 0)
    # print('weee', np.abs((dt * E_inter[1:] * U_half[1:] * S_2[1:]) / V_2[1:]) * np.where(U_half[1:]<0, 1, 0))
    # print('geee', np.abs((dt * E_inter[1:] * U_half[1:] * S) / V))

#     if i==1:
#         print('EEE', E[1,:])
    # Обновление шага по времени, чтобы он отвечал условию КФЛ
    # Если пересчитываем шаг по времени нужно перезапустить этот шаг сначала (делаем индикатор равным False)
    # Если условие ФРЛ продолжает выполняться то делаем индикатор True
    # Если индикатор со значением True выживет до конца шага цикла, то мы должны перейти на следующий временной слой

    # Если производная положительная, то нужно добавить к ячейке что-то из соседней, если отрицательная
#     то нужно вычесть из этой и добавить к соседней
    dt_new = (C * ((log_step * r_phys) * pk_to_meter) / (a + (np.abs(U[1,:])))).min()
    if (dt_new < dt):
        dt = dt_new
        indicator = False
        indicator_2 = True
        print(dt, '&&', i)

        continue
    elif (dt_new >= dt):
#         if (indicator_2 == True):
#             dt = 0.1 * dt_new
        indicator = True
        if indicator_2 == True and i == 1:
            dt = dt_new * dt_minimizer
            indicator_2 = False
            continue
#         elif indicator_2 == True and i>1:
#             dt = dt_new * dt_minimizer_2
#             indicator_2 = False
#             print(dt, '!!', i)
#             continue


    # Обновление давления
    Eps[:] = (E[1,:] - (Rho[1,:]) * U[1,:]**2 / 2) / (Rho[1,:])
    P[1,:] = (gamma - 1) * (Rho[1,:]) * Eps[:]
#     print(P[1,:])
    # Использование граничных условий
#     P[1,-1] = P[1,-2]
#     U[1,-1] = U[1,-2]
#     Eps[-1] = Eps[-2]
#     E[1,-1] = E[1,-2]
#     Rho[1,-1] = Rho[1,-2]

    # Если индикатор равен True, то идем на следующий временной слой, если равен False, то повторяем текущий
    # Записываем в массив все значения шага по времени которые не вызвали пересчет слоя
    if (indicator == True):
        dt_check = np.append(dt_check, dt)
        sim_time = np.sum(dt_check)
#         print(Rho_draw)
        # Сохраняем значения массивов через Draw_step для последующей отрисовки
        if (i%draw_step == 0):
#             print(Rho_draw.shape)
            U_draw = np.vstack([U_draw, U[1,:]])
            Rho_draw = np.vstack([Rho_draw, Rho[1,:]])
#             print(Rho[1, :])
#             print(Rho_draw.shape)
            E_draw = np.vstack([E_draw, E[1,:]])

#             time_0 = time.time() - time_0
            print('Номер шага ', i)
#             print('С предыдущего вывода прошло ', time_0)
            print('Прошло ', sim_time * 3.169e-8, 'лет')
#             print(np.log(U[1,:].max()))

        # Присвоение новых значений массивам
        U[0,:] = U[1,:]
        Rho[0,:] = Rho[1,:]
        E[0,:] = E[1,:]
        P[0,:] = P[1,:]

        U[1,:] = 0
        Rho[1, :] = 0
        E[1, :] = 0
        P[1, :] = 0
#         print(i)
        i = i + 1

    if (i>=1000000):
        break
print('Общее время симуляции равно', np.sum(dt_check) * 3.169e-8, 'лет')

97924.97687575467 && 1
Номер шага  10000
Прошло  37399.51415958529 лет
105431495.06181201 && 11411
103031463.04616982 && 11412
100734086.59237668 && 11413
98460752.95439522 && 11414
96214081.49278457 && 11415
93993548.38894306 && 11416
91798766.31442921 && 11417
89629354.7789867 && 11418
87484945.21033424 && 11419
85365180.65667224 && 11420
83269715.74336198 && 11421
81198216.65509759 && 11422
79150361.15479305 && 11423
77125838.64072314 && 11424
75124350.24419321 && 11425
73145608.97033781 && 11426
71189339.88502699 && 11427
69255280.35127392 && 11428
67343180.3189974 && 11429
65452802.67249877 && 11430
63583923.64057333 && 11431
61736333.27479546 && 11432
59909836.00220142 && 11433
58104251.25935003 && 11434
56319414.21557506 && 11435
54555176.594162025 && 11436
52811407.60118896 && 11437
51087994.972873546 && 11438
49384846.15346756 && 11439
47701889.6170364 && 11440
46039076.34785146 && 11441
44396381.495596856 && 11442
42773806.22313237 && 11443
41171379.7661307 && 11444
39589161.

KeyboardInterrupt: 

In [3]:
# импортируем библиотеку matplotlib
import matplotlib.pyplot as plt

#Задаем начальное значение и шаг по времени для отрисовки графиков
draw_time = 0
draw_time_step = 1

# создаем фигуру и оси
fig, ax = plt.subplots()

# перебираем значения draw_time от 0 до length_t-1 с шагом draw_time_step
for t in range(draw_time, Rho_draw.shape[0], draw_time_step):
    # рисуем график значений Rho[t, :] и r[:] с подписью t
    ax.plot(r_phys, Rho_draw[t, :], label=f"t = {t}")

# добавляем легенду
ax.legend()
# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.set_ylim(2500, 3500)
# ax.set_xlim(10,100)

# показываем фигуру
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
# импортируем библиотеку matplotlib
import matplotlib.pyplot as plt

#Задаем начальное значение и шаг по времени для отрисовки графиков
draw_time = 0
draw_time_step = 5

# создаем фигуру и оси
fig, ax = plt.subplots()

# перебираем значения draw_time от 0 до length_t-1 с шагом draw_time_step
for t in range(draw_time, U_draw.shape[0], draw_time_step):
    # рисуем график значений Rho[t, :] и r[:] с подписью t
    ax.plot(r_phys, U_draw[t, :], label=f"t = {t}")

# добавляем легенду
ax.legend()
# ax.set_ylim(2500, 3500)
# ax.set_xlim(10,12)

# показываем фигуру
plt.show()